# FFT (Fast Fourier Transform)

The following pieces of code will define the FFT, which uses many different algorithms to compute the DFT of a list (or a matrix, which shall be discussed later) of data, defined as:

$X_k = \sum^{N-1}_{n=0}x_n ⋅ e^{-iτkn/N}, \;N ≜ $ size of $x$

(It may also be written as: 
$X_k = \sum^{N-1}_{n=0}x_n ⋅ ω^{kn}, \; N$ ≜ size of $x$, $ω$ ≜ $e^{-iτ/N}$
)

depending on the size of the input list, as follows:

If $N$ is a power of 2, use a radix 2 FFT, else if $N$ is a prime number, use a rader FFT, else use a bluestein FFT.

These algorithms have a big $O$ notation of $O$($ N \cdot log(N) $), whereas manually computing the DFT has a big $O$ notation of $O$($N^2$). (I.e. as $N$ is scaled by some $k$, the time complexity of FFT algortithms increase by $k \cdot log(k)$, whereas the time complexity of a DFT increases by $k^2$).

This makes using these algorithms much more obvious than manual computation of the DFT, hence why they are so useful to us.


The following cell defines some starting code

In [3]:
using Primes

τ = 2π

# Displays a vector after formatting it (makes logs easier to read)
function format_display(x::Union{Vector{T}, Matrix{T}})::Nothing where T <: Real
    display(map(elem -> round(elem, digits=2), x))
end

# Displays a matrix after formatting it (makes logs easier to read)
function format_display(x::Union{Vector{T}, Matrix{T}})::Nothing where T <: Number
    display(map(elem -> round(real(elem), digits=2) + round(imag(x), digits=2)))
end

format_display (generic function with 2 methods)

## Radix 2 FFT 

The following code cell defines a radix 2 FFT

When N is a power of 2, one can rearrange the list by reversing the indices digits (in base 2). One can then recursively compute the FFT butterfly algorithm to reconstruct the desired FFT.

This can be simplified to: 

$FFT(x) = FFT(x_e) + (FFT(x_o) \cdot U^{(N/2)-1}_{n=0}ω^n) \; ⌢ \; FFT(x_e) - (FFT(x_o) \cdot U^{(N/2)-1}_{n=0}ω^n)$

where $U_{r=a}^bf(r) = [f(a), f(a+1), ..., f(b-1), f(b)]$ and $⌢$ is concatenation.

It is thus clear to see that a radix 2 FFT is recursive, and this allows for a much faster computation than manually computing a DFT.

For reasons to be made clear later (see radix 2 IFFT), ω shall be parameterised (to allow different values to be supplied). (By default, it will have a value of nothing, then will be initialised in the function).

In [ ]:
function radix2FFT(x::Vector{T}, ω::Union{ComplexF64, Nothing} = nothing)::Vector{ComplexF64} where T <: Number
    N = length(x)
    
    # ω ≜ exp(-im*τ/N) (if not already supplied) 
    if ω === nothing
        ω = exp(-im*τ/N)
    end    

    Xₑ = radix2FFT(x[1:2:end], ω^2) # Get the even indices of x (0-indexed) and recursively FFT (Julia is 1-indexed)
    Xₒ = radix2FFT(x[2:2:end], ω^2) # Get the odd indices of x (0-indexed) and recursively FFT (Julia is 1-indexed)

    res = zeros(ComplexF64, N)
    
    for i ∈ 1:N÷2
        res[i] = Xₑ[i] + Xₒ[i] * ω^(i-1) 
        res[i+N÷2] = Xₑ[i] - Xₒ[i] * ω^(i-1)
    end

    return res
end